In [1]:
! pip install langchain tiktoken openai pypdf faiss-cpu huggingface_hub python-docx selenium unstructured -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 66.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import SKLearnVectorStore

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from docx import Document
from langchain.document_loaders import SeleniumURLLoader

In [4]:
# Load the PDF file. English or French only

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64,
    separators=['\n\n', '\n', '(?=>\. )', ' ', ''])

pdf_doc_path=''
if pdf_doc_path:
    loader = PyPDFLoader(pdf_doc_path)
    pages = loader.load_and_split()

txt_path=''
if txt_path:
    loader=TextLoader(txt_path)
    pages=loader.load()

docx_path='/content/drive/MyDrive/Data S/data/System engineering.docx'
if docx_path:
  word_doc=Document(docx_path)
  passages=[p.text for p in word_doc.paragraphs]
  content = "\n".join(passages)
  file_txt_path="./file.txt"
  with open(file_txt_path, "w") as f:
    f.write(content)
  loader=TextLoader(file_txt_path)
  pages=loader.load()

url=['']
if url[0]:
  loader = SeleniumURLLoader(url)
  pages=loader.load()

texts = text_splitter.split_documents(pages)
embeddings = OpenAIEmbeddings(openai_api_key="")

In [5]:
vector_db = FAISS.from_documents(texts, embeddings)
vector_db.save_local("./FAISS_db", index_name=("wiki_rmse"))

In [6]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

llm=HuggingFaceHub(huggingfacehub_api_token="",
                    repo_id="tiiuae/falcon-7b-instruct",
                    model_kwargs={"temperature":0.5 ,
                                 "max_length":512,
                                 "max_new_tokens":200
                                 })

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [7]:
from langchain.chains import RetrievalQA

vector_db=FAISS.load_local(folder_path="./FAISS_db",
                           index_name='wiki_rmse',
                           embeddings=embeddings)


qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",
                                 retriever=vector_db.as_retriever(search_kwargs={"k": 2}),
                                 return_source_documents=True,
                                 verbose=False
                                )


In [10]:
#Ask a question?
queary='What is system engineering?'
results=qa({"query": queary})


In [11]:
print(results['result'])


System engineering is a methodical, multi-disciplinary approach for the design, realization, technical management, operations, and retirement of a system. A system is the combination of elements that function together to produce the capability required to meet a need. The elements include all hardware, software, equipment, facilities, personnel, processes, and procedures needed for this purpose; that is, all things required to produce system-level results. The results include system-level qualities, properties, characteristics, functions, behavior, and performance. The value added by the system as a whole, beyond that contributed independently by the parts, is primarily created by the relationship among the parts; that is, how they are interconnected.
